# Guide:
1.   Execute SETUP (1ST TIME).
2.   INSTALL the LIBRARIES for the GENERATION MODELS that you wish to use: AudioLDM2-Music + StableAudio Open share the same libraries. REQUIREMENTS: Login to HuggingFace is necessary to use the aforementioned models (an API key is needed, is free and can be generated here https://huggingface.co/settings/tokens), CUDA drivers and a CUDA compatible architecture is needed (e.g. Google Collabs GPUs, i suggest NVIDIA A100 for music generation inference tasks). Could also work with freely available T4 card, but the memory is almost always not enough to carry out the inference process. MusicGen libraries will take some time to install, much more than the other 2 models.
3. move on UPLOAD: first upload the musical idea on which you want to build on, can be either .mp3 OR .wav (only these 2 formats are acepted for now), and then press Save file, a confirmation and audio preveiw will appear.
4. Advanced Mode will give some more options, for a more professional and personalised result. Normal Mode is for quick test or non-professionals.

BUGS: at the moment it is only possible to install and use EITHER AudioLDM2-Music and StableAudio Open OR MusicGen: If StableAudio and AudioLDM2 libraries are installed it is not possible to install audiocraft beacuse of "NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968". tryed with
"import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')", "!sudo locale-gen en_US.UTF-8
!sudo update-locale", but nothing solved the problem.

In [ ]:
# @title INSTALL AudioLDM2-Music / StableAudio Open Music Generation Model & HuggingFace Login (1ST TIME) {"vertical-output":true,"display-mode":"form"}
# install libraries for the generation model
!pip install diffusers
!pip install torchsde #required by diffusers


# do the login in huggingface
!huggingface-cli login
#HuggingFace key:


#TO-DO: need to make it automatic

In [ ]:
# @title INSTALL MusicGen Music Generation Model (1ST TIME) {"display-mode":"form"}
!pip install audiocraft #takes a while

# SETUP (1ST TIME)

In [ ]:
# install for "user interface"
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.2 MB/s eta 0:00:00


In [ ]:
# install essentia for: bpm detection, key detection

!pip install librosa
# !pip install essentia

In [ ]:
# install essentia-tensorflow for using Essentia models in general

# unistall tensorflow if needed
# !pip uninstall tensorflow -y

!pip install essentia-tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.4/291.4 MB 6.8 MB/s eta 0:00:00


In [ ]:
# download the TempoCNN model from essentia
#YOU CAN DOWNLOAD THE WEIGHTS FOR OTHER TEMPOCNN MODELS HERE, k16 ones are more precise, using more resources:
!wget https://essentia.upf.edu/models/tempo/tempocnn/deepsquare-k16-3.pb
#!wget https://essentia.upf.edu/models/tempo/tempocnn/deeptemp-k4-3.pb
#!wget https://essentia.upf.edu/models/tempo/tempocnn/deeptemp-k16-3.pb

--2024-11-07 23:02:16--  https://essentia.upf.edu/models/tempo/tempocnn/deepsquare-k16-3.pb
Resolving essentia.upf.edu (essentia.upf.edu)... 84.89.139.43
Connecting to essentia.upf.edu (essentia.upf.edu)|84.89.139.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4833690 (4.6M) [application/octet-stream]
Saving to: ‘deepsquare-k16-3.pb’

deepsquare-k16-3.pb 100%[===================>]   4.61M   590KB/s    in 8.7s    

2024-11-07 23:02:26 (540 KB/s) - ‘deepsquare-k16-3.pb’ saved [4833690/4833690]



In [ ]:
# download genre_discogs400 model weights
!wget https://essentia.upf.edu/models/classification-heads/genre_discogs400/genre_discogs400-discogs-effnet-1.pb
!wget https://essentia.upf.edu/models/music-style-classification/discogs-effnet/discogs-effnet-bs64-1.pb
# download MTG-Jamendo genre model weights
!wget https://essentia.upf.edu/models/classification-heads/mtg_jamendo_genre/mtg_jamendo_genre-discogs-effnet-1.pb

# discogs400 metadata (contains genre lables column to decode embeddings)
!wget https://essentia.upf.edu/models/classification-heads/genre_discogs400/genre_discogs400-discogs-effnet-1.json
# mtg_jamendo metadata (contains genre lables column to decode embeddings)
!wget https://essentia.upf.edu/models/classification-heads/mtg_jamendo_genre/mtg_jamendo_genre-discogs-effnet-1.json

--2024-11-07 23:02:26--  https://essentia.upf.edu/models/classification-heads/genre_discogs400/genre_discogs400-discogs-effnet-1.pb
Resolving essentia.upf.edu (essentia.upf.edu)... 84.89.139.43
Connecting to essentia.upf.edu (essentia.upf.edu)|84.89.139.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2057977 (2.0M) [application/octet-stream]
Saving to: ‘genre_discogs400-discogs-effnet-1.pb’

genre_discogs400-di 100%[===================>]   1.96M   472KB/s    in 4.3s    

2024-11-07 23:02:31 (472 KB/s) - ‘genre_discogs400-discogs-effnet-1.pb’ saved [2057977/2057977]

--2024-11-07 23:02:31--  https://essentia.upf.edu/models/music-style-classification/discogs-effnet/discogs-effnet-bs64-1.pb
Resolving essentia.upf.edu (essentia.upf.edu)... 84.89.139.43
Connecting to essentia.upf.edu (essentia.upf.edu)|84.89.139.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18366619 (18M) [application/octet-stream]
Saving to: ‘discogs-effnet-bs64-1

In [ ]:
# Essentia MTG-Jamendo mood and theme (56 classes) [WE ARE NOT GONNA USE THIS MODEL]
# weigths:
#!wget https://essentia.upf.edu/models/classification-heads/mtg_jamendo_moodtheme/mtg_jamendo_moodtheme-discogs-effnet-1.pb
#json for classes:
#!wget https://essentia.upf.edu/models/classification-heads/mtg_jamendo_moodtheme/mtg_jamendo_moodtheme-discogs-effnet-1.json

#embedding model? always the same right?
#!wget https://essentia.upf.edu/models/music-style-classification/discogs-effnet/discogs-effnet-bs64-1.pb

In [ ]:
#Moods MIREX (29 classes in 5 mood clusters)

#vggish
# weigths:
#!wget https://essentia.upf.edu/models/classification-heads/moods_mirex/moods_mirex-audioset-vggish-1.pb
#json for classes:
#!wget https://essentia.upf.edu/models/classification-heads/moods_mirex/moods_mirex-audioset-vggish-1.json
# embedding:
#!wget https://essentia.upf.edu/models/feature-extractors/vggish/audioset-vggish-3.pb

#musicnn
# weigths:
!wget https://essentia.upf.edu/models/classification-heads/moods_mirex/moods_mirex-msd-musicnn-1.pb
#json for classes:
!wget https://essentia.upf.edu/models/classification-heads/moods_mirex/moods_mirex-msd-musicnn-1.json
# embedding:
!wget https://essentia.upf.edu/models/autotagging/msd/msd-musicnn-1.pb

--2024-11-07 23:03:13--  https://essentia.upf.edu/models/classification-heads/moods_mirex/moods_mirex-msd-musicnn-1.pb
Resolving essentia.upf.edu (essentia.upf.edu)... 84.89.139.43
Connecting to essentia.upf.edu (essentia.upf.edu)|84.89.139.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95007 (93K) [application/octet-stream]
Saving to: ‘moods_mirex-msd-musicnn-1.pb’

moods_mirex-msd-mus 100%[===================>]  92.78K   218KB/s    in 0.4s    

2024-11-07 23:03:15 (218 KB/s) - ‘moods_mirex-msd-musicnn-1.pb’ saved [95007/95007]

--2024-11-07 23:03:15--  https://essentia.upf.edu/models/classification-heads/moods_mirex/moods_mirex-msd-musicnn-1.json
Resolving essentia.upf.edu (essentia.upf.edu)... 84.89.139.43
Connecting to essentia.upf.edu (essentia.upf.edu)|84.89.139.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2273 (2.2K) [application/json]
Saving to: ‘moods_mirex-msd-musicnn-1.json’

moods_mirex-msd-mus 100%[=============

In [ ]:
# [WE WON'T USE OPEN AI, IF IMPLEMENT A LLM FOR PROMPT SUGGESTION WILL BE ANOTHER ONE]
# for requesting a suggestion on the instrument to generate,
# based on the extracted data from music_in
# !pip install openai

In [ ]:
# Set the /content working directory
import os
os.chdir("/content")
print("Current working directory:", os.getcwd()) # Verify the current working directory

Current working directory: /content


In [ ]:
##### DEFINITION OF SOME USEFUL FUNCTIONS for extraction algorithms #####

#import json and assign column to list, returns this list: THIS IS USED FOR DECODERS
def list_classes_from_json(file_path, column_name):
  import json
  # Load the classes from the JSON file
  with open(file_path, 'r') as f:
      data = json.load(f)
  # Extract the classes
  list_classes = data[column_name]
  # OUTPUT:
  return list_classes


#process audio for essentia
def processed_audio_essentia(fileaudiopath):
  from essentia.standard import MonoLoader
  import librosa
  # Load your audio file
  y, sr = librosa.load(fileaudiopath)
  # Process the audio
  audio = MonoLoader(filename=fileaudiopath, sampleRate=sr, resampleQuality=4)()
  # OUTPUT:
  return audio

#QUESTO SOTTO MI SA CHE NON SERVE!!!!!!
#creates CSV of column1, column2 ES. filename | result
def create_CSV(column1, column2, filename):
  import csv
  with open(filename, 'w', newline='') as csvfile:
      writer = csv.writer(csvfile)
      for item1, item2 in zip(column1, column2):
        writer.writerow([item1, item2])

In [ ]:
##### BPM EXTRACTION STAGE: TempoCNN (deepsquare-k16-3) #####
#input loop to test with all the files

BPM_model_path = "/content/deepsquare-k16-3.pb" #downloaded in installation stage

def extract_BPM_TempoCNN(file_path):
    from essentia.standard import MonoLoader, TempoCNN

    import librosa
    y, sr = librosa.load(file_path)

    # Load an audio file
    audio = MonoLoader(filename=file_path, sampleRate=sr, resampleQuality=4)()

    model1 = TempoCNN(graphFilename=BPM_model_path)
    global_tempo, local_tempo, local_tempo_probabilities = model1(audio)

    return global_tempo


In [ ]:
##### KEY EXTRACTION STAGE: Essentia KeyExtractor #####
def extract_KEY_KeyExtractor(file_path):
  import essentia.standard as es

  # Load an audio file
  audio = es.MonoLoader(filename=file_path)()

  # Detect the key, scale
  key_detector = es.KeyExtractor()
  key, scale, strength = key_detector(audio)

  return f"{key} {scale} key"

In [ ]:


##### GENRE EXTRACTION STAGE: Essentia discogs400 AND Essentia MTG-Jamendo #####
# model weights and json dowloaded before in installation stage

# Essentia genre_discogs400 (400 classes)
def extract_GENRE_sorted_list_discogs400(file_path):
    # PROCESS:
    import numpy as np
    from essentia.standard import MonoLoader, TensorflowPredictEffnetDiscogs, TensorflowPredict2D
    import librosa
    from collections import Counter

    # Inputs: audio + labels
    audio = processed_audio_essentia(file_path)
    genres = list_classes_from_json("/content/genre_discogs400-discogs-effnet-1.json", 'classes')

    embedding_model = TensorflowPredictEffnetDiscogs(graphFilename="/content/discogs-effnet-bs64-1.pb", output="PartitionedCall:1")
    embeddings = embedding_model(audio)

    model = TensorflowPredict2D(graphFilename="/content/genre_discogs400-discogs-effnet-1.pb", input="serving_default_model_Placeholder", output="PartitionedCall:0")
    predictions = model(embeddings)

    # Collect genre predictions
    time_tags_list = []
    # OUTPUT:
    for i in range(0, len(predictions)):
      time_tags_list.append(genres[np.argmax(predictions[i])])

    # Count occurrences and sort genres by relevance
    time_tags_list_count = Counter(time_tags_list)
    genre_dict_sorted = dict(sorted(time_tags_list_count.items(), key=lambda item: item[1], reverse=True))

    # Extract only the genres in order of relevance as a list
    suggested_genre_list = list(genre_dict_sorted.keys())

    # Exclude specific genres (results from our research)
    excluded_genres = {"Electronic---Ambient", "Electronic---Experimental", "Electronic---Vaporwave", "Electronic---Downtempo"}
    suggested_genre_list = [genre for genre in genre_dict_sorted.keys() if genre not in excluded_genres]

    return suggested_genre_list



# Essentia MTG-Jamendo genre (87 classes)
def extract_GENRE_sorted_list_mtg_jamendo(file_path):

    # PROCESS:
    import numpy as np
    import librosa
    from essentia.standard import MonoLoader, TensorflowPredictEffnetDiscogs, TensorflowPredict2D
    from collections import Counter


    audio = processed_audio_essentia(file_path)
    genres = list_classes_from_json("/content/mtg_jamendo_genre-discogs-effnet-1.json", 'classes')


    embedding_model = TensorflowPredictEffnetDiscogs(graphFilename="/content/discogs-effnet-bs64-1.pb", output="PartitionedCall:1")
    embeddings = embedding_model(audio)

    model = TensorflowPredict2D(graphFilename="mtg_jamendo_genre-discogs-effnet-1.pb")
    predictions = model(embeddings)

    # Collect genre predictions
    time_tags_list = []
    # OUTPUT:
    for i in range(0, len(predictions)):
      time_tags_list.append(genres[np.argmax(predictions[i])])

    # Count occurrences and sort genres by relevance
    time_tags_list_count = Counter(time_tags_list)
    genre_dict_sorted = dict(sorted(time_tags_list_count.items(), key=lambda item: item[1], reverse=True))

    # Extract only the genres in order of relevance as a list
    suggested_genre_list = list(genre_dict_sorted.keys())


    # Exclude specific genres (results from our research)
    excluded_genres = {"ambient", "classical"}
    suggested_genre_list = [genre for genre in genre_dict_sorted.keys() if genre.lower() not in excluded_genres]


    return suggested_genre_list


In [ ]:


##### MOOD EXTRACTION STAGE: Essentia MTG-Jamendo AND Essentia MIREX musicnn #####

# Mood model for MTG-Jamendo [DONT USE IT!!!]
def extract_MOOD_sorted_list_MTGJamendo(file_path):
    import numpy as np
    from essentia.standard import MonoLoader, TensorflowPredictEffnetDiscogs, TensorflowPredict2D
    from collections import Counter

    # Load audio and mood classes
    audio = processed_audio_essentia(file_path)
    moods = list_classes_from_json("/content/mtg_jamendo_moodtheme-discogs-effnet-1.json", 'classes')

    # Load and run embedding and mood prediction models
    embedding_model = TensorflowPredictEffnetDiscogs(graphFilename="/content/discogs-effnet-bs64-1.pb", output="PartitionedCall:1")
    embeddings = embedding_model(audio)

    model = TensorflowPredict2D(graphFilename="/content/mtg_jamendo_moodtheme-discogs-effnet-1.pb", input="serving_default_model_Placeholder", output="PartitionedCall:0")
    predictions = model(embeddings)

    # Collect predictions and count occurrences
    time_tags_list = [moods[np.argmax(pred)] for pred in predictions]
    time_tags_list_count = Counter(time_tags_list)
    mood_dict_sorted = dict(sorted(time_tags_list_count.items(), key=lambda item: item[1], reverse=True))

    # Filter to include only tags from mood_tags_filtered_list
    mood_tags_filtered_list = [
        "calm", "cool", "dream", "emotional", "energetic",
        "fun", "funny", "groovy", "happy", "heavy", "hopeful", "inspiring",
        "love", "meditative", "melancholic", "melodic", "motivational",
        "positive", "powerful", "relaxing", "romantic", "sad", "sexy", "soft",
        "upbeat", "uplifting"
    ]
    filtered_mood_list = [tag for tag in mood_dict_sorted.keys() if tag in mood_tags_filtered_list]

    return filtered_mood_list



# Mood model for MIREX musicnn [USE THIS]
def extract_MOOD_sorted_list_MIREX_musicnn(file_path):
    import numpy as np
    from essentia.standard import MonoLoader, TensorflowPredictMusiCNN, TensorflowPredict2D
    from collections import Counter

    # Load audio and mood classes
    audio = processed_audio_essentia(file_path)
    #moods = list_classes_from_json("/content/moods_mirex-msd-musicnn-1.json", 'classes')
    #these mood tags are too complex, we are going to use a new list that retain the same significance but is more functional for a music generation model:
    moods = ["Bold, Energetic", "Playful, Friendly", "Thoughtful, Melancholic", "whimsical, silly", "Intense, Fierce"]

    # Load and run embedding and mood prediction models
    embedding_model = TensorflowPredictMusiCNN(graphFilename="/content/msd-musicnn-1.pb", output="model/dense/BiasAdd")
    embeddings = embedding_model(audio)

    model = TensorflowPredict2D(graphFilename="/content/moods_mirex-msd-musicnn-1.pb", input="serving_default_model_Placeholder", output="PartitionedCall")
    predictions = model(embeddings)

    # Collect predictions and count occurrences
    time_tags_list = [moods[np.argmax(pred)] for pred in predictions]
    time_tags_list_count = Counter(time_tags_list)
    mood_dict_sorted = dict(sorted(time_tags_list_count.items(), key=lambda item: item[1], reverse=True))

    return list(mood_dict_sorted.keys())


In [ ]:


##### some functions for generation models #####

#calculate the correct time-lenght for the loop given the bpm
def audio_duration_BPM(bpm, beats):
  n_beats = 32
  one_beat_duration_given_bpm = 60/bpm
  audio_duration = one_beat_duration_given_bpm * n_beats
  return audio_duration



def show_audio_box(file_path):
  print("\nHere is your new music loop! Coherent with your musical idea, ready to drag in the DAW.")
  from IPython.display import display, Audio
  display(Audio(file_path))



def show_download_button(file_path):
    import ipywidgets as widgets
    from IPython.display import display

    # Create a button widget
    download_button = widgets.Button(description="Download", button_style='success')

    # Define the function to run when the button is clicked
    def on_button_click(b):
        from google.colab import files
        files.download(file_path)

    # Attach the function to the button
    download_button.on_click(on_button_click)

    # Display the button
    display(download_button)









##### GENERATION MODELS #####

def music_generation_AudioLDM2(prompt, num_inference_steps, audio_duration):
  # AudioLDM2-Music w/ diffusers
  # https://huggingface.co/cvssp/audioldm2-music

  #generate
  from diffusers import AudioLDM2Pipeline
  import torch

  repo_id = "cvssp/audioldm2-music"
  pipe = AudioLDM2Pipeline.from_pretrained(repo_id, torch_dtype=torch.float16)
  pipe = pipe.to("cuda")

  audio = pipe(prompt, num_inference_steps=num_inference_steps, audio_length_in_s=audio_duration).audios[0]

  #save
  import scipy
  filename = f'[AudioLDM2-Music] {prompt}.wav'
  scipy.io.wavfile.write(filename, rate=16000, data=audio)

  show_audio_box(filename)

  show_download_button(filename)






def music_generation_StableAudioOpen(prompt, num_inference_steps, audio_duration, negative_prompt):
  # StableAudio Open w/ diffusers

  # PROCESS:
  import torch
  import soundfile as sf
  from diffusers import StableAudioPipeline

  pipe = StableAudioPipeline.from_pretrained("stabilityai/stable-audio-open-1.0")
  pipe = pipe.to("cuda")

  # create random seed for generator
  import random
  random_seed = random.randint(0, 9999)
  generator = torch.Generator("cuda").manual_seed(random_seed)

  # run the generation
  audio = pipe(
      prompt,
      negative_prompt=negative_prompt,
      num_inference_steps=num_inference_steps,
      audio_end_in_s = audio_duration, #audio duration calculated in the step before!!
      num_waveforms_per_prompt=3,
      generator=generator,
  ).audios

  # OUTPUT:
  output = audio[0].T.float().cpu().numpy()
  filename = f'[StableAudio Open] {prompt} [seed:{random_seed}].wav'
  sf.write(filename, output, pipe.vae.sampling_rate)

  show_audio_box(filename)

  show_download_button(filename)




def music_generation_MusicGenStereoMedium(prompt, audio_duration):
# MusicGen: musicgen-stereo-medium
  import torchaudio
  from audiocraft.models import MusicGen
  from audiocraft.data.audio import audio_write

  model = MusicGen.get_pretrained('facebook/musicgen-stereo-medium')
  model.set_generation_params(duration=audio_duration)
  description = [prompt]
  wav = model.generate(description)

  for idx, one_wav in enumerate(wav):
      # Will save with loudness normalization at -14 db LUFS.
      filename = f'[musicgen-stereo-medium] {prompt}'
      audio_write(filename, one_wav.cpu(), model.sample_rate, strategy="loudness", loudness_compressor=True)

      show_audio_box(filename + ".wav")

      show_download_button(filename)


In [ ]:
!pip install --upgrade numba librosa

# PIPELINE

In [ ]:
# @title UPLOAD {"vertical-output":true,"display-mode":"form"}

from ipywidgets import FileUpload, Button
from IPython.display import display

# Create a FileUpload widget that accepts only one file
upload = FileUpload(accept='.mp3, .wav', multiple=False)  # Accepts .mp3 and .wav files only

# Function to handle button clicks
def save_file(b):
    # Save locally the uploaded file as "music_in"
    if upload.value:
        for name, file_info in upload.value.items():
            content = file_info['content']  # Binary file data
            with open("music_in.wav", 'wb') as f:
                f.write(content)
            print(f"File '{name}' has been saved to disk as 'music_in.wav'.")

            from IPython.display import display, Audio
            display(Audio(input_music_in_path))

# Button
button_save_file = Button(description="Save file")
button_save_file.on_click(save_file)

# Display the upload widget and the button
display(upload)
display(button_save_file)

# INPUT STAGE
input_music_in_path = '/content/music_in.wav' #music_in path




In [ ]:
# @title Advanced Mode execution {"display-mode":"form"}




# INPUT STAGE
input_music_in_path = '/content/music_in.wav' #music_in path
# input_prompt will be taken in the UI
input_negative_prompt = "Low quality." # what u dont want from the generation, use "Low quality." as default








############### PART 1: FEATURE EXTRACTION ##############

BPM = extract_BPM_TempoCNN(input_music_in_path)

KEY = extract_KEY_KeyExtractor(input_music_in_path)


# Extract the music genres from both the models
GENRE_disco = extract_GENRE_sorted_list_discogs400(input_music_in_path)
GENRE_jamendo = extract_GENRE_sorted_list_mtg_jamendo(input_music_in_path)
# All the genre tags
GENRE = GENRE_disco + GENRE_jamendo


# MOOD = extract_MOOD_sorted_list_MTGJamendo(input_music_in_path)
MOOD = extract_MOOD_sorted_list_MIREX_musicnn(input_music_in_path)











############### PART 2: CREATE PRE-FILLED UI ##############
import ipywidgets as widgets
from IPython.display import display

# Sample lists of options for genre and mood
genre_options = GENRE
mood_options = MOOD

# Default values for genre and mood
# TO-DO: should adjust with Try, since if one of the list is empty, this will give error
default_genre = genre_options[0]  # first element
default_mood = mood_options[0]  # first element


# Define the input fields
input_prompt = widgets.Text(
    value='synth pluck arpeggio, high pitched',
    description='PROMPT:',
    disabled=False
)

bpm_input = widgets.IntText(
    value=BPM,  # default BPM value
    description='BPM:',
    disabled=False
)

# button x2 bpm
duplicate_button = widgets.Button(
    description='x2',
    disabled=False,
    layout=widgets.Layout(width='48px')
)
def duplicate_bpm(b):
    bpm_input.value = bpm_input.value * 2
duplicate_button.on_click(duplicate_bpm)


# button /2 bpm
divide_button = widgets.Button(
    description='/2',
    disabled=False,
    layout=widgets.Layout(width='48px')
)
def divide_bpm(b):
    bpm_input.value = bpm_input.value / 2
divide_button.on_click(divide_bpm)



n_beats_input = widgets.IntText(
    value=32,  # default number of beats to generate
    description='N. BEATS:',
    disabled=False
)

key_input = widgets.Text(
    value=KEY,  # default key
    description='KEY:',
    disabled=False
)


# Allow the user to input a custom genre
genre_input = widgets.Text(
    value=default_genre,  # pre-fill with first element of the list
    description='GENRE:',
    placeholder='Type custom genre here...',
    disabled=False
)

# Allow the user to input a custom mood
mood_input = widgets.Text(
    value=default_mood,  # pre-fill with first element of the list
    description='MOOD:',
    placeholder='Type custom mood here...',
    disabled=False
)

# Define the genre selection with dropdown
genre_dropdown = widgets.Dropdown(
    options=genre_options,
    value=default_genre,  # default genre
    description='',
    disabled=False,
    layout=widgets.Layout(width='100px')
)

# Define the mood selection with dropdown
mood_dropdown = widgets.Dropdown(
    options=mood_options,
    value=default_mood,  # default mood
    description='',
    disabled=False,
    layout=widgets.Layout(width='100px')
)

# Define the model options
model_options = widgets.ToggleButtons(
    options=['StableAudio Open', 'MusicGen', 'AudioLDM2-Music'],
    description='MODEL:',
    button_style='',
    tooltips=['StableAudio Open model', 'MusicGen model', 'AudioLDM2-Music model']
)

inference_steps_slider = widgets.IntSlider(
    value=40,  # default value
    min=10,     # minimum value
    max=400,    # maximum value
    step=1,     # step size
    description='',
    continuous_update=False,  # update on release
    layout=widgets.Layout(width='500px')
)

slider_box = widgets.HBox([
    widgets.Label(value='Inference Steps:', layout=widgets.Layout(width='120px')),
    inference_steps_slider
])

# Model descriptions
model_descriptions = {
    'StableAudio Open': "(DEFAULT) StableAudio Open is the best model, allowing for hi-res audio at 44100 kHz. It excels at generating modern, single-instrument tracks, particularly for electronic music, but struggles with older niche/uncommon acoustic instruments and vintage sounds.",
    'MusicGen': "MusicGen offers lower quality than StableAudio, struggles with single instrument generation but handles vintage sounds better. It occasionally generates glitches, which may be interesting for experimental audio loops. MusicGen don't take Infernce Steps as input.",
    'AudioLDM2-Music': "AudioLDM2 has the lowest quality, often producing audio that sounds distant, somewhat uncanny, as if from a post-apocalyptic radio. This model is interesting for experimental contexts, but it struggles with single instrument clarity."
}

# Description area for model info
model_description = widgets.Output()

# Function to update model description based on selection
def update_model_description(change):
    model_description.clear_output()
    with model_description:
        print(model_descriptions[change['new']])

# Bind model options to description update
model_options.observe(update_model_description, names='value')

# Update the text box when a selection is made from the dropdown
def update_genre(change):
    genre_input.value = change['new']

def update_mood(change):
    mood_input.value = change['new']

# Bind the dropdown change events to update functions
genre_dropdown.observe(update_genre, names='value')
mood_dropdown.observe(update_mood, names='value')

# function to run when the GENERATE button is clicked:
def generate_output(b):
    genre = genre_input.value if genre_input.value else genre_dropdown.value
    mood = mood_input.value if mood_input.value else mood_dropdown.value
    print(f"Generating {input_prompt.value}\nBPM: {bpm_input.value}\nKey: {key_input.value}\nGenre: {genre}\nMood: {mood}")

    # audio duration
    audio_duration = audio_duration_BPM(bpm_input.value, n_beats_input.value)
    print(f"Audio duration: {audio_duration}")

    # creazione del prompt
    def prompt_complete(prompt, genre, mood, key, bpm):
      return f"{prompt}, solo, {genre}, {mood}, {key}, {bpm} BPM"
    prompt = prompt_complete(input_prompt.value, genre_input.value, mood_input.value, key_input.value, bpm_input.value)
    print(prompt)

    # inference steps
    num_inference_steps = inference_steps_slider.value
    print(num_inference_steps)


    #SELEZIONE DEL MODELLO, RICHIAMO DELLA FUNZIONE DEL MODELLO, GENERAZIONE
    generation_model_selection = model_options.value
    print(generation_model_selection)

    if generation_model_selection == "MusicGen":
      music_generation_MusicGenStereoMedium(prompt, audio_duration)
    elif generation_model_selection == "AudioLDM2-Music":
      music_generation_AudioLDM2(prompt, num_inference_steps, audio_duration)
    elif generation_model_selection == "StableAudio Open":
      music_generation_StableAudioOpen(prompt, num_inference_steps, audio_duration, input_negative_prompt)




# Create the GENERATE button
generate_button = widgets.Button(description="GENERATE")
generate_button.on_click(generate_output)

# Create horizontal layouts for genre and mood
genre_hbox = widgets.HBox([genre_input, genre_dropdown])
mood_hbox = widgets.HBox([mood_input, mood_dropdown])

#bpm box to duplicate or divide
bpm_hbox = widgets.HBox([bpm_input, duplicate_button, divide_button])

# Display all the widgets
display(input_prompt, bpm_hbox, n_beats_input, key_input, genre_hbox, mood_hbox, model_options, model_description, slider_box, generate_button)

# Initialize description with the default selection
with model_description:
    print(model_descriptions[model_options.value])



In [ ]:
# @title Normal Mode execution {"display-mode":"form"}




# INPUT STAGE
input_music_in_path = '/content/music_in.wav' #music_in path
# input_prompt will be taken in the UI
input_negative_prompt = "Low quality." # what u dont want from the generation, use "Low quality." as default










############### PART 1: FEATURE EXTRACTION ##############

BPM = extract_BPM_TempoCNN(input_music_in_path)

KEY = extract_KEY_KeyExtractor(input_music_in_path)

# Assign genre considering that the list could be empty
GENRE_jamendo_list = extract_GENRE_sorted_list_mtg_jamendo(input_music_in_path)
try: GENRE = GENRE_jamendo_list[0]
except IndexError: GENRE = ""
# print(GENRE)

# Assign mood considering that the list could be empty
MOOD_mirex_list = extract_MOOD_sorted_list_MIREX_musicnn(input_music_in_path)
try: MOOD = MOOD_mirex_list[0]
except IndexError: MOOD = ""
# print(MOOD)












############### PART 2: CREATE PRE-FILLED UI ##############
import ipywidgets as widgets
from IPython.display import display


# Define the input fields
input_prompt = widgets.Text(
    value='synth pluck arpeggio, high pitched',
    description='PROMPT:',
    disabled=False
)

bpm_input = widgets.IntText(
    value=BPM,  # default BPM value
    description='BPM:',
    disabled=False
)

# button x2 bpm
duplicate_button = widgets.Button(
    description='x2',
    disabled=False,
    layout=widgets.Layout(width='48px')
)
def duplicate_bpm(b):
    bpm_input.value = bpm_input.value * 2
duplicate_button.on_click(duplicate_bpm)


# button /2 bpm
divide_button = widgets.Button(
    description='/2',
    disabled=False,
    layout=widgets.Layout(width='48px')
)
def divide_bpm(b):
    bpm_input.value = bpm_input.value / 2
divide_button.on_click(divide_bpm)



n_beats_input = widgets.IntText(
    value=32,  # default number of beats to generate
    description='N. BEATS:',
    disabled=False
)

key_input = widgets.Text(
    value=KEY,  # default key
    description='KEY:',
    disabled=False
)


inference_steps_slider = widgets.IntSlider(
    value=40,  # default value
    min=10,     # minimum value
    max=400,    # maximum value
    step=1,     # step size
    description='',
    continuous_update=False,  # update on release
    layout=widgets.Layout(width='500px')
)

slider_box = widgets.HBox([
    widgets.Label(value='Inference Steps:', layout=widgets.Layout(width='120px')),  # Adjust width as needed
    inference_steps_slider
])


# Define the function to run when the GENERATE button is clicked
def generate_output(b):
    genre = GENRE
    mood = MOOD
    print(f"Generating {input_prompt.value}\nBPM: {bpm_input.value}\nKey: {key_input.value}\nGenre: {genre}\nMood: {mood}")

    # audio duration
    audio_duration = audio_duration_BPM(bpm_input.value, n_beats_input.value)
    print(f"Audio duration: {audio_duration}")

    num_inference_steps=50

    # creazione del prompt
    def prompt_complete(prompt, genre, mood, key, bpm):
      return f"{prompt}, solo, {genre}, {mood}, {key}, {bpm} BPM"
    prompt = prompt_complete(input_prompt.value, GENRE, MOOD, key_input.value, bpm_input.value)
    print(prompt)

    music_generation_StableAudioOpen(prompt, num_inference_steps, audio_duration, input_negative_prompt)




# Create the GENERATE button
generate_button = widgets.Button(description="GENERATE")
generate_button.on_click(generate_output)


#bpm box to duplicate or divide
bpm_hbox = widgets.HBox([bpm_input, duplicate_button, divide_button])

# Display all the widgets
display(input_prompt, bpm_hbox, n_beats_input, key_input, generate_button)


